In [8]:
import py3Dmol

def read_gro_file_with_box_and_units(filename):
    """
    Reads atomic positions and box dimensions from a .gro file.
    Converts units from nanometers to angstroms for py3Dmol visualization.
    Returns:
        - gro_data: The entire contents of the .gro file as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    box_dims = (0.0, 0.0, 0.0)
    try:
        with open(filename, "r") as file:
            gro_data = file.read()
            
            # Extract the last line for box dimensions
            lines = gro_data.strip().split("\n")
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                # Convert box dimensions from nm to Å
                box_dims = tuple(float(dim) * 10 for dim in box_line[:3])
                print(f"Extracted box dimensions (in Å): {box_dims}")
            else:
                print("Box dimensions line is invalid.")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    return gro_data, box_dims

def visualize_gro_with_box_units(gro_data, box_dims):
    """
    Visualizes the .gro file with simulation box edges using py3Dmol.
    Converts the box dimensions and coordinates to angstroms.
    Args:
        - gro_data: The .gro file contents as a string.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    if box_dims == (0.0, 0.0, 0.0):
        print("Invalid box dimensions. Cannot draw simulation box.")
        return

    viewer = py3Dmol.view(width=800, height=600)
    
    # Add the molecular structure from the .gro file
    viewer.addModel(gro_data, "gro")  # Load .gro file
    viewer.setStyle({"sphere": {"scale": 0.3}})  # Atoms as spheres
    
    # Add the box edges (in Å)
    x_len, y_len, z_len = box_dims
    box_lines = [
        [[0, 0, 0], [x_len, 0, 0]], [[x_len, 0, 0], [x_len, y_len, 0]],
        [[x_len, y_len, 0], [0, y_len, 0]], [[0, y_len, 0], [0, 0, 0]],  # Bottom face
        [[0, 0, z_len], [x_len, 0, z_len]], [[x_len, 0, z_len], [x_len, y_len, z_len]],
        [[x_len, y_len, z_len], [0, y_len, z_len]], [[0, y_len, z_len], [0, 0, z_len]],  # Top face
        [[0, 0, 0], [0, 0, z_len]], [[x_len, 0, 0], [x_len, 0, z_len]],
        [[x_len, y_len, 0], [x_len, y_len, z_len]], [[0, y_len, 0], [0, y_len, z_len]]  # Vertical edges
    ]
    for line in box_lines:
        viewer.addCylinder({"start": {"x": line[0][0], "y": line[0][1], "z": line[0][2]},
                            "end": {"x": line[1][0], "y": line[1][1], "z": line[1][2]},
                            "radius": 0.1, "color": "gray"})

    # Center and zoom the viewer
    viewer.zoomTo()
    
    # Show the viewer
    return viewer.show()

def main():
    # Input .gro file
    gro_filename = "/home/newsitt/Desktop/topic1/dtac_isolated/conf.gro"
    
    # Read the .gro file and box dimensions
    gro_data, box_dims = read_gro_file_with_box_and_units(gro_filename)
    
    # Visualize the .gro file with the simulation box
    visualize_gro_with_box_units(gro_data, box_dims)

if __name__ == "__main__":
    main()

Extracted box dimensions (in Å): (30.0, 30.0, 30.0)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
import py3Dmol
import os
import math

def read_gro_file_with_box_and_units(filename):
    """
    Reads atomic positions and box dimensions from a .gro file.
    Converts units from nanometers to angstroms for py3Dmol visualization.
    Returns:
        - atoms: A list of atom information (residue, atom name, x, y, z in Å).
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    atoms = []
    box_dims = (0.0, 0.0, 0.0)
    
    try:
        with open(filename, "r") as file:
            lines = file.readlines()
            
            # Atom information starts after the second line and ends before the last line
            for line in lines[2:-1]:
                # Parse .gro atom line format
                res_num = int(line[:5].strip())  # Residue number
                res_name = line[5:10].strip()   # Residue name
                atom_name = line[10:15].strip() # Atom name
                x, y, z = [float(line[i:i+8].strip()) * 10 for i in range(20, 44, 8)]  # Convert nm to Å
                atoms.append((res_num, res_name, atom_name, x, y, z))
            
            # Extract the last line for box dimensions
            box_line = lines[-1].split()
            if len(box_line) >= 3:
                box_dims = tuple(float(dim) * 10 for dim in box_line[:3])  # Convert nm to Å
                print(f"Extracted box dimensions (in Å): {box_dims}")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except ValueError as ve:
        print(f"Error while reading the file: {ve}")
    
    return atoms, box_dims

def calculate_bonds(atoms, bond_threshold=1.6):
    """
    Calculates bonds between atoms based on a distance threshold.
    Returns a list of bonds as pairs of atom indices.
    """
    bonds = []
    for i, atom1 in enumerate(atoms):
        for j, atom2 in enumerate(atoms):
            if i < j:
                x1, y1, z1 = atom1[3:6]
                x2, y2, z2 = atom2[3:6]
                distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
                if distance <= bond_threshold:
                    bonds.append((i, j))
    return bonds

def visualize_gro_with_bonds(atoms, bonds, box_dims):
    """
    Visualizes the .gro file with bonds and simulation box using py3Dmol.
    Args:
        - atoms: List of atom information (residue, atom name, x, y, z in Å).
        - bonds: List of bonds as pairs of atom indices.
        - box_dims: Dimensions of the simulation box in angstroms as (x, y, z).
    """
    viewer = py3Dmol.view(width=800, height=600)
    
    # Add atoms as spheres
    for i, atom in enumerate(atoms):
        _, _, atom_name, x, y, z = atom
        viewer.addSphere({
            "center": {"x": x, "y": y, "z": z},
            "radius": 0.3,
            "color": "blue"  # You can customize colors based on atom type
        })
    
    # Add bonds as cylinders
    for bond in bonds:
        atom1 = atoms[bond[0]]
        atom2 = atoms[bond[1]]
        x1, y1, z1 = atom1[3:6]
        x2, y2, z2 = atom2[3:6]
        viewer.addCylinder({
            "start": {"x": x1, "y": y1, "z": z1},
            "end": {"x": x2, "y": y2, "z": z2},
            "radius": 0.1,
            "color": "gray"
        })
    
    # Add simulation box if valid
    if box_dims != (0.0, 0.0, 0.0):
        x_len, y_len, z_len = box_dims
        box_lines = [
            [[0, 0, 0], [x_len, 0, 0]], [[x_len, 0, 0], [x_len, y_len, 0]],
            [[x_len, y_len, 0], [0, y_len, 0]], [[0, y_len, 0], [0, 0, 0]],
            [[0, 0, z_len], [x_len, 0, z_len]], [[x_len, 0, z_len], [x_len, y_len, z_len]],
            [[x_len, y_len, z_len], [0, y_len, z_len]], [[0, y_len, z_len], [0, 0, z_len]],
            [[0, 0, 0], [0, 0, z_len]], [[x_len, 0, 0], [x_len, 0, z_len]],
            [[x_len, y_len, 0], [x_len, y_len, z_len]], [[0, y_len, 0], [0, y_len, z_len]]
        ]
        for line in box_lines:
            viewer.addCylinder({"start": {"x": line[0][0], "y": line[0][1], "z": line[0][2]},
                                "end": {"x": line[1][0], "y": line[1][1], "z": line[1][2]},
                                "radius": 0.1, "color": "gray"})

    viewer.zoomTo()
    return viewer.show()

def main():
    # Input .gro file
    gro_filename = "/mnt/new_volume/newsitt/topic1/slab/box.gro"
    
    # Read the .gro file
    atoms, box_dims = read_gro_file_with_box_and_units(gro_filename)
    
    # Calculate bonds based on a threshold
    bonds = calculate_bonds(atoms, bond_threshold=1.6)
    
    # Visualize the structure with bonds and box
    visualize_gro_with_bonds(atoms, bonds, box_dims)

if __name__ == "__main__":
    main()


Error while reading the file: invalid literal for int() with base 10: 'F'


3Dmol.js failed to load for some reason. Please check your browser console for error messages.